## Using LLM with Kendra 

Before running this notebook, you will need to first run the notebook titled `deploy_openchatkit_on_sagemaker.ipynb` so that the model is deployed to an endpoint first. In this notebook, we will use the model to do the inference. 

We will first get the response from Kendra and feed it to the model to get a more precise answer to the query

In [2]:
!pip install -U sagemaker --quiet

In [3]:
import boto3
import json
import sagemaker

s3=boto3.resource('s3')
region = boto3.session.Session().region_name
role = sagemaker.get_execution_role()

s3Bucket = sagemaker.Session().default_bucket()
smr_client = boto3.client("sagemaker-runtime")

In [4]:
# Define Kendra client
kendra = boto3.client('kendra')
indexId = '884609ed-c06c-452f-9383-b708df745995'   #remember to change to your index ID here

In [138]:
query='What is SageMaker Ground Truth'

In [139]:
response=kendra.query(
        QueryText = query,
        IndexId = indexId, # paste the Index ID here
)

In [140]:
for query_result in response['ResultItems']:
        
        if query_result['Type']=='QUESTION_ANSWER':
            document_text = query_result['AdditionalAttributes'][1]['Value']['TextWithHighlightsValue']['Text']
            break

        elif query_result['Type']=='ANSWER':
            document_text = query_result['AdditionalAttributes'][0]['Value']['TextWithHighlightsValue']['Text']
            break
            
        elif query_result['Type']=='DOCUMENT':
            document_text = query_result['DocumentExcerpt']['Text']

In [141]:
context = document_text.replace("\n","")
question = query

In [142]:
context

'Amazon SageMaker Amazon SageMaker is a fully-managed machine learning (ML) service that enables developers and data scientists to quickly and easily build, train, and deploy machine learning models at any scale. Amazon SageMaker Ground Truth helps build training data sets quickly and accurately using an active learning model to label data, combining machine learning and human interaction to make the model progressively better.  SageMaker provides fully-managed and pre-built Jupyter notebooks to address common use cases. The services come with multiple built-in, high-performance algorithms, and there is the AWS Marketplace for Machine Learning containing more than 100 additional pre-trained ML models and algorithms. You can also bring your own algorithms and frameworks that are built into a Docker container.'

In [143]:
# Get the endpoint from the notebook "deploy_openchatkit_on_sagemaker"
endpoint_name = 'gpt-neox-djl20-acc-2023-03-28-03-40-52-368-endpoint'

In [144]:
%%time

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": ["Answer the question and summarise it in one line. Question: "+ question + "Context: "+ context + "Answer: "],
            "parameters": {
                "num_return_sequences": 1, 
                "max_new_tokens": 200, 
                "temperature": 0.7, 
                "top_k": 5,
        }
        }))

response_LLM = json.loads(response_model["Body"].read().decode("utf8"))

CPU times: user 16.5 ms, sys: 3.86 ms, total: 20.4 ms
Wall time: 47.4 s


In [146]:
import re

answer_match = re.search(r'Answer:\s*(.*)', response_LLM["outputs"][0])

if answer_match:
    answer = answer_match.group(1)
    pattern = r'[a-zA-Z]'
    match = re.search(pattern, answer)
    
    if match:
        answer_text = answer
    else:
        answer_text = "No answer found in the output"
else:
    answer_text = "No answer found in the output"

print(answer_text)

SageMaker Ground Truth is a fully-managed service that helps build training data sets quickly and accurately using an active learning model to label data, combining machine learning and human interaction to make the model progressively better.
